<div class="info">
<b>PROBLEM SHEET 10:</b>    ipywidgets to give GUI controls to your Jupyter Notebooks.

### Standard Header
As we will be utilizing a number of packages with reasonably long names, we will adopt the _de facto_ standard module abbreviations in the following header.  We also ensure that our [division behavior is sensible](http://www.python.org/dev/peps/pep-0238/) by importing from `__future__`:  _i.e._, promotion to `double` will occur from `int` or `long` data types involving division:  `1/2 == 0.5`.  Although this is the default in Python 3, it is a trivial way to help this notebook work in Python 2 if that's what you are using.

In [ ]:
%matplotlib notebook
#this allows interactive view 
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import math
# Let printing work the same in Python 2 and 3
from __future__ import division,print_function
# notice two underscores _ either side of future


As demonstrated in the lecture `ipywidgets` allows users to visualize and manipulate their data in intuitive and easy ways.
* Researchers can easily see how changing inputs to a model impacts the results. 
* Scientists can share interactive results with graphical user interfaces that others can play with without seeing code. 
* Exploring, learning, and sharing becomes a more immersive experience.

In this final problem sheet you will bring together all of your new skills to produce an interactive (with `ipywidgets`) jupyter notebook that explains a physics / maths concept to the user (choose from list below, or suggest your own).

Whichever topic you chose to explore, the end product must be a self-contained program that when executed explains to the user the physical concepts that it aims to demonstrate. In addition the functionality of the program, what buttons, sliders etc do, should be made clear to the user, through the addition of explanatory text boxes or some form of help function. The user should not be confronted with a series of inputs *n, x, m, etc* without some description of what the parameters mean, possibly some default or suggested values that best demonstrate the physics.

The program will be assessed with the following criteria:

* How clearly is a physics concept explained.
* Does the program work, do all of the functions work as described, how robust is the program.
* Visually is the interface well laid out, does it look interesting, are buttons, plots etc displayed in logical positions.
* How clear is the code, does it follow standard conventions, are functions and classes well commented, do functions, classes and variables have sensible and logical names.
* Have you demonstrated a wide range of programming skills.
    - Have you used the most appropriate graphical representation of the data.
    - Where graphs are used do they have legends, annotations, appropriate axis labels,titles, ticks etc.
    - Depending on the topic, have you demonstrated the use of special functions (Bessel, Hermite, etc).
    - Can the GUI read data from a file, or save the output to a file.
    - Demonstration of curve fitting to compare numerical and analytical solutions.
* You are free to embellish the GUI with any other functionality that you feel would aid in understanding the phenomena.

In addition to the code within the *markdown* cells you should write a report to describe the physics that you are representing and a description of its functionality of the code.
* The report should be fairly compact, I suggest no more than four pages, unless the complexity of the code requires a lengthier description.
* The relevant equations, with all parameters and variables described should be included in the report, in reports with multiple equations you should enumerate them.
* Any figures should have a caption that adequately describes the figure and the should be referred to from the main body of the text.
* References to material that you used to research the topic should be included in the bibliography (even if this is just the course notes to PH2130, PH2210, PH2420 etc.).

Some helpful syntax for markdown can be found here:
https://notebook.community/mathnathan/notebooks/notes/Markdown%20Cheatsheet



## Choice of Topics:

### Quantum Mechanics:
Consider a program that generates the wavefunction and probability densities for different energy levels of the hydrogen atom (or possible the quantum harmonic oscillator). Can you demonstrate orthogonality, normalisation etc through the
use of numerical integration. Could you demonstrate the time evolution of the <span>Schr&ouml;dinger<span> equation? Can you plot the Spherical harmonics for different quantum numbers.
You can use https://www.st-andrews.ac.uk/physics/quvis/ for inspiration of what you could try to display in you code. 

### Fourier Series:
Demonstrate how various functions can be simulated with fourier series, animate the evolution as a function of the order of the series.


### Double Pendulum:
Evaluate different methods of numerical differentiation Euler method, Runge Kutta method by for example modelling the motion of a pendulum, the pendulum example can be extended to include damping, or additional elements e.g. double pendulum, inverted pendulum etc.

### Drum modes: 

Use `scipy.special` to visulalise the vibrational modes on a 2D circular membrane using Bessel functions. Allow the user to select the two mode indices to see each mode. Consider animating a mode. Possibily extend to rectangular or square membranes. 

In [35]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from scipy.optimize import curve_fit


class Derivatives:
    """
    Calculate derivatives for the simple pendulum equations

    Parameters:
    - theta (float): Current angular position of the pendulum
    - omega (float): Current angular velocity of the pendulum
    - L (float): Length of the pendulum
    - g (float): Acceleration due to gravity
    - damping (float): Damping factor affecting the motion
    - driving_force (float): External driving force applied

    Returns:
    - float: The derivative of angular velocity with respect to time
    """

    def __init__(self, theta, omega, L, g, damping, driving_force):
        self.theta = theta
        self.omega = omega
        self.L = L
        self.g = g
        self.damping = damping
        self.driving_force = driving_force

    def domega_dt(self):
        """
        Calculate the derivative of angular velocity with respect to time

        Returns:
        - float: The derivative of angular velocity with respect to time
        """
        return -(np.sin(self.theta) * (self.g / self.L)) - (self.damping * self.omega) + self.driving_force

    def d_dt(self):
        """
        Calculate both derivatives: dtheta_dt and domega_dt

        Returns:
        - tuple: Tuple containing the derivatives dtheta_dt and domega_dt
        """
        dtheta_dt = self.omega
        domega_dt = -((self.g / self.L) * np.sin(self.theta)) - (self.damping * self.omega) + self.driving_force
        return dtheta_dt, domega_dt

class SimplePendulum:
    """
    Model a simple pendulum and solve its motion equations using Euler and Runge-Kutta methods

    Parameters:
    - theta_0 (float): Initial angular position of the pendulum
    - omega_0 (float): Initial angular velocity of the pendulum
    - L (float): Length of the pendulum
    - n (int): Number of time steps for simulation
    - damping (float): Damping factor affecting the motion
    - driving_force (float): External driving force applied

    Returns:
    - object: Instance of SimplePendulum class
    """

    def __init__(self, theta_0, omega_0, L, n, damping, driving_force):
        self.theta_0 = theta_0
        self.omega_0 = omega_0
        self.L = L
        self.n = n
        self.dt = 0.01
        self.g = 9.81
        self.time_ = np.zeros(self.n)
        self.omega_ = np.zeros(self.n)
        self.theta_ = np.zeros(self.n)
        self.omega_[0] = self.omega_0
        self.theta_[0] = self.theta_0 * np.pi / 180.0
        self.damping = damping
        self.driving_force = driving_force

    def euler_method(self):
        """
        Solve the pendulum motion equations using the Euler method

        Returns:
        - object: Instance of SimplePendulum class with updated values
        """
        for i in range(self.n - 1):
            self.time_[i + 1] = self.time_[i] + self.dt
            self.omega_[i + 1] = self.omega_[i] + self.dt * Derivatives(self.theta_[i], self.omega_[i], self.L, self.g,
                                                                        self.damping, self.driving_force).domega_dt()
            self.theta_[i + 1] = self.theta_[i] + self.dt * self.omega_[i]
        return self

    def runge_kutta_method(self):
        """
        Solve the pendulum motion equations using the Runge-Kutta method

        Returns:
        - object: Instance of SimplePendulum class with updated values
        """
        k1_theta, k1_omega = Derivatives(self.theta_[0], self.omega_[0], self.L, self.g, self.damping,
                                         self.driving_force).d_dt()

        for i in range(self.n - 1):
            self.time_[i + 1] = self.time_[i] + self.dt

            k2_theta, k2_omega = Derivatives(self.theta_[i] + 0.5 * self.dt * k1_theta,
                                             self.omega_[i] + 0.5 * self.dt * k1_omega, self.L, self.g, self.damping,
                                             self.driving_force).d_dt()

            k3_theta, k3_omega = Derivatives(self.theta_[i] + 0.5 * self.dt * k2_theta,
                                             self.omega_[i] + 0.5 * self.dt * k2_omega, self.L, self.g, self.damping,
                                             self.driving_force).d_dt()

            k4_theta, k4_omega = Derivatives(self.theta_[i] + self.dt * k3_theta,
                                             self.omega_[i] + self.dt * k3_omega, self.L, self.g, self.damping,
                                             self.driving_force).d_dt()

            self.theta_[i + 1] = self.theta_[i] + (self.dt / 6) * (k1_theta + 2 * k2_theta + 2 * k3_theta + k4_theta)
            self.omega_[i + 1] = self.omega_[i] + (self.dt / 6) * (k1_omega + 2 * k2_omega + 2 * k3_omega + k4_omega)

            k1_theta, k1_omega = Derivatives(self.theta_[i + 1], self.omega_[i + 1], self.L, self.g, self.damping,
                                             self.driving_force).d_dt()
            

        return self


class PendulumAnimate:
    """
    Animate the pendulum motion using Matplotlib animations

    Parameters:
    - theta_0 (float): Initial angular position of the pendulum
    - omega_0 (float): Initial angular velocity of the pendulum
    - L (float): Length of the pendulum
    - n (int): Number of time steps for simulation
    - damping (float): Damping factor affecting the motion
    - driving_force (float): External driving force applied

    Returns:
    - object: Instance of PendulumAnimate class
    """

    def __init__(self, theta_0, omega_0, L, n, damping, driving_force):
        self.theta_0 = theta_0
        self.omega_0 = omega_0
        self.L = L
        self.n = n
        self.damping = damping
        self.driving_force = driving_force
        self.dt = 0.01
        self.g = 9.81

    def pendulum_position(self, theta):
        """
        Calculate the position (x, y) of the pendulum bob

        Parameters:
        - theta (float): Angular position of the pendulum

        Returns:
        - tuple: Tuple containing the x and y coordinates of the pendulum bob
        """
        x = self.L * np.sin(theta)
        y = -self.L * np.cos(theta)
        return x, y

    def initial_pendulum(self):
        """
        Set up the initial pendulum configuration for animation
        """
        self.fig = plt.figure()
        self.ax = self.fig.add_subplot(aspect='equal')
        x_0, y_0 = self.pendulum_position(self.theta_0)
        self.line, = self.ax.plot([0, x_0], [0, y_0], lw=3, c='k')
        pend_radius = self.L * 0.06
        self.circle = self.ax.add_patch(plt.Circle(self.pendulum_position(self.theta_0), pend_radius,
                                                   fc='r', zorder=3))
        self.ax.set_xlim(-self.L * 1.2, self.L * 1.2)
        self.ax.set_ylim(-self.L * 1.2, self.L * 1.2)

    def animate(self, i):
        """
        Animation function for Euler method
        """
        theta = SimplePendulum(self.theta_0, self.omega_0, self.L, self.n, self.damping,
                                self.driving_force).euler_method().theta_
        x, y = self.pendulum_position(theta[i])
        self.line.set_data([0, x], [0, y])
        self.circle.set_center((x, y))

    def animate_plot(self):
        """
        Animate the pendulum motion using Euler method
        """
        ani = animation.FuncAnimation(self.fig, self.animate, frames=self.n, repeat=True,
                                      interval=self.dt * 1000)
        html_video = HTML(ani.to_jshtml())
        display(html_video)
        plt.close()

    def animate_rk4(self, i):
        """
        Animation function for Runge-Kutta method
        """
        theta = SimplePendulum(self.theta_0, self.omega_0, self.L, self.n, self.damping,
                               self.driving_force).runge_kutta_method().theta_
        x, y = self.pendulum_position(theta[i])
        self.line.set_data([0, x], [0, y])
        self.circle.set_center((x, y))

    def animate_plot_rk4(self):
        """
        Animate the pendulum motion using Runge-Kutta method
        """
        ani = animation.FuncAnimation(self.fig, self.animate_rk4, frames=self.n, repeat=True,
                                      interval=self.dt * 1000)
        html_video = HTML(ani.to_jshtml())
        display(html_video)
        plt.close()

In [43]:
def plot_comparison(theta, omega, L, n, damping, driving_force, show_plot=True):
    pendulum_euler = SimplePendulum(theta, omega, L, n, damping, driving_force)
    pendulum_euler.euler_method()

    pendulum_rk4 = SimplePendulum(theta, omega, L, n, damping, driving_force)
    pendulum_rk4.runge_kutta_method()

    
    if show_plot:
        fig, ax = plt.subplots(figsize=(10, 6))
        plt.plot(pendulum_euler.time_, pendulum_euler.theta_ * (180 / np.pi), label='Euler Method')
        plt.plot(pendulum_rk4.time_, pendulum_rk4.theta_ * (180 / np.pi), label='Runge-Kutta Method')
        
        plt.title('Pendulum Motion - Euler vs Runge-Kutta')
        plt.xlabel('Time (s)')
        plt.ylabel('Angle (degrees)')
        plt.legend()
        plt.grid(True)
        
        # Calculate relative error
        relative_error = np.abs(pendulum_euler.theta_ - pendulum_rk4.theta_) / np.abs(pendulum_rk4.theta_)
    
        # Print maximum relative error
        relative_error = np.average(relative_error)
        print(f"Average Relative Error: {relative_error:.4f}")

def update(theta=45, omega=0, L=1, n=1500, damping=0, driving_force=0, show_plot=True):
    """
    Update and display results for both methods to compare the two.

    Parameters:
    - theta (float): Initial angle in degrees
    - omega (float): Initial angular velocity
    - L (float): Length of the pendulum
    - n (int): Number of time steps
    - damping (float): Damping coefficient
    - driving_force (float): External driving force
    - show_plot (bool): Flag to control whether to display the plot

    Returns:
    - Tuple of updated parameters: (theta, omega, L, n, damping, driving_force)
    """

    plot_comparison(theta, omega, L, n, damping, driving_force, show_plot)
    return theta, omega, L, n, damping, driving_force


# Dropdown widget for method selection
method_selector = widgets.Dropdown(
    options=[' ', 'Euler', 'Runge Kutta'],
    value=' ',
    description='Methods:'
)

# Output container for sliders
container_sliders = widgets.Output()


def on_method_change(change):
    """
    Handle changes when a different method is selected

    Parameters:
    - change (dict): Dictionary containing information about the change
    """
    with container_sliders:
        # Clear previous output
        clear_output(wait=True)

        # Display sliders for Euler method
        if change['new'] == 'Euler':
            display(widgets.interact_manual(update,
                                            theta=(0, 90, 1),
                                            omega=(0, 0.5, 0.01), 
                                            L=(0, 10, 0.1), 
                                            n=(100, 3000, 10), 
                                            damping=(0, 0.5, 0.01), 
                                            driving_force=(0, 1, 0.01),
                                            show_plot=widgets.fixed(True)))
            # Create and display Euler animation and comparison data
            theta_0, omega_0, L, n, damping, driving_force = update(show_plot=False)
            plt.rcParams['animation.embed_limit'] = 2 ** 128
            pendulum = PendulumAnimate(theta_0, omega_0, L, n, damping, driving_force)
            pendulum.initial_pendulum()
            pendulum.animate_plot()
            

        elif change['new'] == 'Runge Kutta':
            # Display sliders for Runge-Kutta method
            display(widgets.interact_manual(update,
                                            theta=(0, 90, 1), 
                                            omega=(0, 0.5, 0.01), 
                                            L=(0, 10, 0.1),
                                            n=(100, 3000, 10), 
                                            damping=(0, 0.5, 0.01), 
                                            driving_force=(0, 1, 0.01),
                                            show_plot=widgets.fixed(True)))
            # Create and display Runge-Kutta animation and comparison data
            theta_0, omega_0, L, n, damping, driving_force = update(show_plot=False)
 
            pendulum = PendulumAnimate(theta_0, omega_0, L, n, damping, driving_force)
            pendulum.initial_pendulum()
            pendulum.animate_plot_rk4()


# Observe method selector changes
method_selector.observe(on_method_change, names='value')

# Display method selector and sliders container
display(method_selector)
display(container_sliders)

# Trigger initial update based on the default method
on_method_change({'new': method_selector.value})

Dropdown(description='Methods:', options=(' ', 'Euler', 'Runge Kutta'), value=' ')

Output()

>## Dynamic Showcase Of The Simple Pendulum
>### Alexander Boxer PH2150 PS10 Report
>#### December 9, 2023

### 1 Abstract

The study of classical mechanics gives us insight to how a pendulum oscillates in the real world, wether that be with a damping and/or driving force the motion of a pendulum can be measured using Newtons laws of motion. In this report I will demonstrate two methods of estimating the motion of a pendulum given different circumstances that simulate the real world motion of the pendulum. Using ipywidgets the input values can be dynamic and change as requested to display a range of circumstances in which a pendulum may oscillate within.

### 2 Theory

It is widely known from Newton's second law of motion that \(F = m \cdot a\), this gives us the basis in which we can start to construct our equation for the motion of the pendulum. To derive the motion of a free pendulum, we require two forces acting upon the pendulum:
1. The force of gravity
2. The tension in the string

![Pendulum Forces](attachment:force_diagram.png)

Fig 1 - Pendulum diagram depicting the forces acting upon it in free motion [1]
* $\(mg\)$: Force of gravity acting downward on the mass.
* $\(T\)$: Tension in the string.

The net force acting on the mass $F_{\text{net}}$ can be expressed as the vector sum of these forces: $$F_{\text{net}} = T - mg$$
This net force is responsible for the acceleration of the mass: $$a(t) = \frac{d^{2}\theta}{d^{2}t}$$
Resolving these forces in the $x$ and $y$ direction we retrieve: $$ F_{x} = -T * sin(\theta) $$ $$F_{y} = T * cos(\theta) - mg$$
Substituting this into $F = ma$ we can recover the differential equation for a pendulum in free motion [1]: $$\theta '' = -\frac{g}{L}sin(\theta)$$
By including a damping term which is proportional to the angular velocity of the pendulum and a constant driving force we can recover a full differential equation for the motion of the pendulum: $$\theta '' = - \frac{\beta}{m}\theta ' -\frac{g}{L}sin(\theta) + F_{d}$$ We can assume small angles of $\theta$ for $sin(\theta) \approx \theta$ this gives us our final differential equation: $$\theta '' + \frac{\beta}{m}\theta ' + \frac{g}{L}\theta = F_{d}$$ With this differential equation we can use the trial solution $\theta = e^{r\theta}$ and $\theta_p = \frac{L}{g} F_d$ as the particular integral for this nonhomogeneous differential equation: $$\theta = e^{at}(Asin(bt) + Bcos(bt)) + \frac{L}{g}F_d$$ Finding that $$ r = a + bi$$ $$a = -\frac{1}{2} \frac{\beta}{m}$$ $$b = \frac{1}{2}\sqrt{(\frac{\beta}{m})^{2} - 4\frac{g}{L}}$$
Where:
* $\beta$ = Damping constant
* $\omega$ = $\frac{d\theta}{dt}$
* $F_{d}$ = Driving force constant

With this information at hand I am tasked with solving this differential equation through two numerical methods, this required me to use the Euler and Runge Kutta method in my approach to determine the angular displacement and velocity of the pendulum. For the Euler method the general solution [2] $$\theta_{i}=\theta + dt * \frac{d^{2}\theta}{dt^{2}}$$ where $dt$ is a small change in time $(t)$. To compare this I used the Runge Kutta method, this involves using a step at the midpoint of an interval $(dt)$ to cancel out lower order error terms. The general formula used is [2] $$k_{1} = dt * f(\theta_n, \omega_n)$$ $$k_2 = dt * f(\theta_n + \frac{1}{2}dt, \omega_n + \frac{1}{2}k_1)$$ $$k_3 = dt * f(\theta_n + \frac{1}{2}dt, \omega_n + \frac{1}{2}k_2)$$ $$k_4 = dt * f(\theta_n + dt, \omega_n + k_3)$$ $$\theta_{n+1} = \theta_n + dt * (\frac{1}{6}k_1 + \frac{1}{3}k_2 + \frac{1}{3}k_3 + \frac{1}{6}k_4)$$ With these formulae we can now solve for the angular velocity and displacement of the pendulum.

### 3 Back end code

The back end is dedicated to handle the calculations of data and return tuples to be processed by the front end functions. The back end contains three classes all with its own unique function of which plays a role in preparing data for visulalisation on the front end. The main parameters we will be working with are:
* $theta_0$ (float): Initial angular position of the pendulum
* $omega_0$ (float): Initial angular velocity of the pendulum
* $L$ (float): Length of the pendulum
* $n$ (int): Number of time steps for simulation
* $damping$ (float): Damping factor affecting the motion
* $driving_force$ (float): External driving force applied

These variables are what the user can change through a slider on the front end (more on that later) these variables define the initial conditions and conditions in which the pendulum oscillates in which in turn determine further values of the angular displacement $(\theta)$ and angular velocity $(\omega)$. Calling upon the function *derivatives* with the initial conditions will be able to calculate the angular acceleration and set the constant terms within the differential equation. 

The *SimplePendulum* class contains methods to solve for the $\omega$ and $\theta$ therms, these include: 1. the Euler method and 2. the Runge Kutta method . Both methods within the *SimplePendulum* class will return values for $\theta$ and $\omega$ in a tuple which can be used to graphically represent the methods an enable a comparison between the three methods. In both numerical methods an empty list of length $n$ is created using the *np.zeros* command, which creates the empty lists *time_, omega_, theta_* all of which will contain the values of their respective names for all instances of zero. These tuples are then used within the iterative methods of numerical differentiation to return a full list each containing their respective values for all values of n.

The *PendulumAnimate* class takes use of the matplotlib.animation library where it is possible to create a live animation of the data that is calculated in *SimplePendulum* visually. This is able to plot the results of both the Euler and Runge Kutta method. Breaking the class down by each method. The *pendulum_position* method returns the Cartesian coordinates of the pendulum due to the fact that *matplotlib* displays plots in Cartesian coordinates, this uses the conversion from polar to Cartesian equations $$x = r * sin(\theta)$$ $$y = r * sin(\theta)$$ since we would like the pendulum to be at the bottom of the plot $y$ becomes $-y$ so it is more in line with real world observations. The *initial_pendulum* method dictates the initial position of the pendulum and plots a pendulum bob and string onto a graph in the initial position it also initialises the size and shape of the plot in which the animation will be in. The *animate* methods use the numerical methods to return a tuples containing the values of theta, these values are converted into cartesian coordinates and used to update the position of the pendulum bob and string. The *animate_plot* methods use the matplotlib.animation.FuncAnimation to retrieve all frames of the pendulum and convert it into a usable animation format which can be displayed on the front end.

### 4 Frond end code

The front end code contains all the necessary elements to present a user friendly interface that can be used to visualise the different methods of solving the pendulum equation. This cell, once executed, will display a drop-down in which the user can select between the types of method that will be displayed and animated, this drop down also updates the output and can change from Euler to Runge Kutta animations, for example, as requested. Once a method is selected the user will be shown a slider, here they will select the initial conditions in which the pendulum will oscillate, **Theta_0, Omega_0, L, n, damping, and driving force**. Once selected from a provided interval for each term, the user is prompted to run the code using a toggle button, once pressed a graph representing the movement of the pendulum will appear depicting both numerical methods, an animation of the pendulum with being solved using the selected method in the drop down, a value will be outputted as the average relative error between the two methods to determine how similar they are over the time interval selecetd.

### References
[1] Chakraborty A. Differential Equations; Accessed 10/12/2023. Available from: https://www.isical.ac.in/~arnabc/numana/diff1.html#:~:text=We%20can%20write%20the%20differential,y%E2%80%B2(t)).

[2] Casey A. PH2150 ODE. Accessed: 10/12/2023 Available from: PH2150 Moodle. 


In [ ]:
#ignore this, it's something that helps styling the notebook.
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()